In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
from skimage.exposure import equalize_adapthist
from skimage.restoration import denoise_tv_chambolle

def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        
        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())
        
        img = np.dstack([r,g,b])
        img = denoise_tv_chambolle(img,weight=0.2,multichannel=True)
        #img = std_img(img)
        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [6]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.0008
    elif epoch<30:
        return 0.0004
    else:
        return 0.0001
    

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.57653, saving model to best_m.h5
 - 2s - loss: 0.6471 - acc: 0.6011 - val_loss: 0.5765 - val_acc: 0.6729
Epoch 2/150
Epoch 00002: val_loss improved from 0.57653 to 0.46463, saving model to best_m.h5
 - 1s - loss: 0.5820 - acc: 0.6990 - val_loss: 0.4646 - val_acc: 0.7975
Epoch 3/150
Epoch 00003: val_loss improved from 0.46463 to 0.45657, saving model to best_m.h5
 - 2s - loss: 0.4821 - acc: 0.7758 - val_loss: 0.4566 - val_acc: 0.8474
Epoch 4/150
Epoch 00004: val_loss improved from 0.45657 to 0.43237, saving model to best_m.h5
 - 2s - loss: 0.4972 - acc: 0.7628 - val_loss: 0.4324 - val_acc: 0.8536
Epoch 5/150
Epoch 00005: val_loss improved from 0.43237 to 0.36567, saving model to best_m.h5
 - 2s - loss: 0.4432 - acc: 0.7977 - val_loss: 0.3657 - val_acc: 0.8629
Epoch 6/150
Epoch 00006: val_loss improved from 0.36567 to 0.32417, saving model to best_m.h5
 - 2s - loss: 0.4178 - acc: 0.8073 - val_loss: 0.3242 - val_acc: 0.8442
Epoch 7/

Epoch 61/150
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.2358 - acc: 0.9029 - val_loss: 0.2230 - val_acc: 0.9003
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2457 - acc: 0.8857 - val_loss: 0.2157 - val_acc: 0.8910
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2186 - acc: 0.9172 - val_loss: 0.2173 - val_acc: 0.8910
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2355 - acc: 0.9000 - val_loss: 0.2208 - val_acc: 0.8847
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2370 - acc: 0.9008 - val_loss: 0.2240 - val_acc: 0.8879
Epoch 66/150
Epoch 00066: val_loss improved from 0.21540 to 0.21272, saving model to best_m.h5
 - 2s - loss: 0.2135 - acc: 0.9180 - val_loss: 0.2127 - val_acc: 0.8879
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2358 - acc: 0.9091 - val_loss: 0.2236 - val_acc: 0.8910
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2312 - acc: 0.9039 - val_los

Epoch 126/150
Epoch 00126: val_loss did not improve
 - 1s - loss: 0.2204 - acc: 0.9164 - val_loss: 0.2147 - val_acc: 0.9065
Epoch 127/150
Epoch 00127: val_loss did not improve
 - 1s - loss: 0.1865 - acc: 0.9258 - val_loss: 0.2180 - val_acc: 0.8941
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 1s - loss: 0.2045 - acc: 0.9172 - val_loss: 0.2063 - val_acc: 0.9065
Epoch 129/150
Epoch 00129: val_loss did not improve
 - 1s - loss: 0.2054 - acc: 0.9141 - val_loss: 0.2115 - val_acc: 0.8910
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 2s - loss: 0.1974 - acc: 0.9258 - val_loss: 0.2161 - val_acc: 0.9003
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 1s - loss: 0.1931 - acc: 0.9250 - val_loss: 0.2066 - val_acc: 0.8972
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 1s - loss: 0.2041 - acc: 0.9094 - val_loss: 0.2219 - val_acc: 0.8972
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 2s - loss: 0.1933 - acc: 0.9273 - val_loss: 0.2134 - val_acc: 0.8941
Epoch 13

Epoch 38/150
Epoch 00038: val_loss did not improve
 - 1s - loss: 0.2284 - acc: 0.9086 - val_loss: 0.2514 - val_acc: 0.9003
Epoch 39/150
Epoch 00039: val_loss did not improve
 - 1s - loss: 0.2476 - acc: 0.9000 - val_loss: 0.2587 - val_acc: 0.9003
Epoch 40/150
Epoch 00040: val_loss improved from 0.25002 to 0.24068, saving model to best_m.h5
 - 1s - loss: 0.2315 - acc: 0.8966 - val_loss: 0.2407 - val_acc: 0.9065
Epoch 41/150
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.2416 - acc: 0.9047 - val_loss: 0.2469 - val_acc: 0.8972
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.2082 - acc: 0.9086 - val_loss: 0.2606 - val_acc: 0.8785
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.2476 - acc: 0.8886 - val_loss: 0.2492 - val_acc: 0.8879
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.2035 - acc: 0.9156 - val_loss: 0.2651 - val_acc: 0.8910
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.2489 - acc: 0.8945 - val_los

Epoch 104/150
Epoch 00104: val_loss did not improve
 - 1s - loss: 0.2083 - acc: 0.9140 - val_loss: 0.2508 - val_acc: 0.8941
Epoch 105/150
Epoch 00105: val_loss did not improve
 - 1s - loss: 0.2047 - acc: 0.9115 - val_loss: 0.2496 - val_acc: 0.8941
Epoch 106/150
Epoch 00106: val_loss did not improve
 - 1s - loss: 0.1754 - acc: 0.9242 - val_loss: 0.2616 - val_acc: 0.8879
Epoch 107/150
Epoch 00107: val_loss did not improve
 - 1s - loss: 0.1861 - acc: 0.9172 - val_loss: 0.2373 - val_acc: 0.8972
Epoch 108/150
Epoch 00108: val_loss did not improve
 - 1s - loss: 0.2051 - acc: 0.9164 - val_loss: 0.2393 - val_acc: 0.9003
Epoch 109/150
Epoch 00109: val_loss did not improve
 - 1s - loss: 0.1800 - acc: 0.9258 - val_loss: 0.2734 - val_acc: 0.9034
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 1s - loss: 0.1824 - acc: 0.9281 - val_loss: 0.2429 - val_acc: 0.9003
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 1s - loss: 0.2089 - acc: 0.9115 - val_loss: 0.2638 - val_acc: 0.8879
Epoch 11

...

Epoch 146/150
Epoch 00146: val_loss did not improve
 - 1s - loss: 0.2051 - acc: 0.9224 - val_loss: 0.1589 - val_acc: 0.9408
Epoch 147/150
Epoch 00147: val_loss did not improve
 - 1s - loss: 0.1792 - acc: 0.9344 - val_loss: 0.1570 - val_acc: 0.9377
Epoch 148/150
Epoch 00148: val_loss did not improve
 - 2s - loss: 0.2226 - acc: 0.9146 - val_loss: 0.1822 - val_acc: 0.9346
Epoch 149/150
Epoch 00149: val_loss did not improve
 - 1s - loss: 0.1907 - acc: 0.9211 - val_loss: 0.1648 - val_acc: 0.9221
Epoch 150/150
Epoch 00150: val_loss did not improve
 - 1s - loss: 0.1976 - acc: 0.9209 - val_loss: 0.1549 - val_acc: 0.9439
Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.59859, saving model to best_m.h5
 - 2s - loss: 0.6717 - acc: 0.5818 - val_loss: 0.5986 - val_acc: 0.7259
Epoch 2/150
Epoch 00002: val_loss improved from 0.59859 to 0.47236, saving model to best_m.h5
 - 2s - loss: 0.5829 - acc: 0.6993 - val_loss: 0.4724 - val_acc: 0.7944
Epoch 3/150
Epoch 00003: val_loss improved from 0.47

Epoch 56/150
Epoch 00056: val_loss did not improve
 - 1s - loss: 0.2459 - acc: 0.8937 - val_loss: 0.2220 - val_acc: 0.9065
Epoch 57/150
Epoch 00057: val_loss did not improve
 - 1s - loss: 0.2482 - acc: 0.8922 - val_loss: 0.2205 - val_acc: 0.9128
Epoch 58/150
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.2356 - acc: 0.9039 - val_loss: 0.2230 - val_acc: 0.8972
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.2308 - acc: 0.8964 - val_loss: 0.2286 - val_acc: 0.8972
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.2567 - acc: 0.8797 - val_loss: 0.2285 - val_acc: 0.9003
Epoch 61/150
Epoch 00061: val_loss improved from 0.21923 to 0.21477, saving model to best_m.h5
 - 2s - loss: 0.2467 - acc: 0.8961 - val_loss: 0.2148 - val_acc: 0.9065
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.2237 - acc: 0.9037 - val_loss: 0.2174 - val_acc: 0.9097
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2412 - acc: 0.9000 - val_los

Epoch 00121: val_loss did not improve
 - 2s - loss: 0.2271 - acc: 0.9031 - val_loss: 0.2243 - val_acc: 0.9003
Epoch 122/150
Epoch 00122: val_loss did not improve
 - 1s - loss: 0.2207 - acc: 0.9130 - val_loss: 0.2146 - val_acc: 0.9065
Epoch 123/150
Epoch 00123: val_loss did not improve
 - 2s - loss: 0.2020 - acc: 0.9242 - val_loss: 0.2161 - val_acc: 0.9034
Epoch 124/150
Epoch 00124: val_loss did not improve
 - 2s - loss: 0.2061 - acc: 0.9273 - val_loss: 0.2292 - val_acc: 0.8910
Epoch 125/150
Epoch 00125: val_loss did not improve
 - 2s - loss: 0.2070 - acc: 0.9156 - val_loss: 0.2616 - val_acc: 0.8847
Epoch 126/150
Epoch 00126: val_loss improved from 0.20729 to 0.20427, saving model to best_m.h5
 - 2s - loss: 0.2050 - acc: 0.9195 - val_loss: 0.2043 - val_acc: 0.9003
Epoch 127/150
Epoch 00127: val_loss did not improve
 - 2s - loss: 0.2343 - acc: 0.8951 - val_loss: 0.2259 - val_acc: 0.8972
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 2s - loss: 0.2072 - acc: 0.9068 - val_loss: 0.2

Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2301 - acc: 0.9047 - val_loss: 0.3385 - val_acc: 0.8562
Epoch 34/150
Epoch 00034: val_loss did not improve
 - 2s - loss: 0.2327 - acc: 0.9016 - val_loss: 0.3421 - val_acc: 0.8656
Epoch 35/150
Epoch 00035: val_loss did not improve
 - 2s - loss: 0.2049 - acc: 0.9117 - val_loss: 0.3429 - val_acc: 0.8688
Epoch 36/150
Epoch 00036: val_loss did not improve
 - 2s - loss: 0.2482 - acc: 0.8984 - val_loss: 0.3478 - val_acc: 0.8594
Epoch 37/150
Epoch 00037: val_loss did not improve
 - 2s - loss: 0.2139 - acc: 0.8984 - val_loss: 0.3427 - val_acc: 0.8500
Epoch 38/150
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.2264 - acc: 0.9039 - val_loss: 0.3506 - val_acc: 0.8531
Epoch 39/150
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.2092 - acc: 0.9078 - val_loss: 0.3407 - val_acc: 0.8562
Epoch 40/150
Epoch 00040: val_loss improved from 0.32954 to 0.32914, saving model to best_m.h5
 - 2s - loss: 0.2423 - acc: 0.8891 - val_loss: 0.3291 - v

Epoch 99/150
Epoch 00099: val_loss did not improve
 - 2s - loss: 0.1877 - acc: 0.9164 - val_loss: 0.3399 - val_acc: 0.8656
Epoch 100/150
Epoch 00100: val_loss did not improve
 - 2s - loss: 0.1998 - acc: 0.9226 - val_loss: 0.3446 - val_acc: 0.8688
Epoch 101/150
Epoch 00101: val_loss did not improve
 - 1s - loss: 0.1867 - acc: 0.9187 - val_loss: 0.3393 - val_acc: 0.8781
Epoch 102/150
Epoch 00102: val_loss did not improve
 - 2s - loss: 0.1967 - acc: 0.9188 - val_loss: 0.3374 - val_acc: 0.8750
Epoch 103/150
Epoch 00103: val_loss did not improve
 - 2s - loss: 0.1963 - acc: 0.9133 - val_loss: 0.3447 - val_acc: 0.8750
Epoch 104/150
Epoch 00104: val_loss did not improve
 - 2s - loss: 0.2026 - acc: 0.9156 - val_loss: 0.3598 - val_acc: 0.8625
Epoch 105/150
Epoch 00105: val_loss did not improve
 - 2s - loss: 0.1912 - acc: 0.9258 - val_loss: 0.3658 - val_acc: 0.8656
Epoch 106/150
Epoch 00106: val_loss did not improve
 - 2s - loss: 0.2011 - acc: 0.9258 - val_loss: 0.3294 - val_acc: 0.8781
Epoch 107

In [7]:
import pickle
with open('../features/cnn_1_aug_skimage_denoise_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

#pre 3219
# new 2965

# skimage 2725

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug_denoise_preprocess.csv', index=False)

0.223606538315
         id  is_iceberg
0  5941774d    0.203983
1  4023181e    0.306734
2  b20200e4    0.001557
3  e7f018bb    0.986334
4  4371c8c3    0.000734


In [8]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [9]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.54269, saving model to best_m.h5
 - 3s - loss: 0.6360 - acc: 0.6140 - val_loss: 0.5427 - val_acc: 0.7757
Epoch 2/150
Epoch 00002: val_loss improved from 0.54269 to 0.44427, saving model to best_m.h5
 - 2s - loss: 0.5403 - acc: 0.7382 - val_loss: 0.4443 - val_acc: 0.8380
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4904 - acc: 0.7828 - val_loss: 0.4746 - val_acc: 0.8131
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.4309 - acc: 0.8219 - val_loss: 0.5142 - val_acc: 0.7352
Epoch 5/150
Epoch 00005: val_loss improved from 0.44427 to 0.34409, saving model to best_m.h5
 - 2s - loss: 0.3649 - acc: 0.8388 - val_loss: 0.3441 - val_acc: 0.8505
Epoch 6/150
Epoch 00006: val_loss improved from 0.34409 to 0.33525, saving model to best_m.h5
 - 2s - loss: 0.3764 - acc: 0.8375 - val_loss: 0.3353 - val_acc: 0.8380
Epoch 7/150
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3692 - acc: 0.8398 - val_loss:

Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1862 - acc: 0.9234 - val_loss: 0.2618 - val_acc: 0.8785
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1855 - acc: 0.9211 - val_loss: 0.2666 - val_acc: 0.8816
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1691 - acc: 0.9258 - val_loss: 0.2443 - val_acc: 0.9034
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1911 - acc: 0.9201 - val_loss: 0.2529 - val_acc: 0.8941
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1766 - acc: 0.9242 - val_loss: 0.2613 - val_acc: 0.8847
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1928 - acc: 0.9242 - val_loss: 0.2420 - val_acc: 0.8910
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1773 - acc: 0.9246 - val_loss: 0.2493 - val_acc: 0.8941
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1793 - acc: 0.9226 - val_loss: 0.2464 - val_acc: 0.8972
Epoch 71/150
Epoch 00071: val

Epoch 130/150
Epoch 00130: val_loss did not improve
 - 1s - loss: 0.1302 - acc: 0.9484 - val_loss: 0.3188 - val_acc: 0.8660
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 1s - loss: 0.1382 - acc: 0.9469 - val_loss: 0.3170 - val_acc: 0.8785
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 1s - loss: 0.1426 - acc: 0.9430 - val_loss: 0.3031 - val_acc: 0.8723
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 1s - loss: 0.1319 - acc: 0.9476 - val_loss: 0.2974 - val_acc: 0.8816
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 1s - loss: 0.1167 - acc: 0.9539 - val_loss: 0.3112 - val_acc: 0.8723
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 1s - loss: 0.1367 - acc: 0.9430 - val_loss: 0.2964 - val_acc: 0.8660
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 1s - loss: 0.1477 - acc: 0.9422 - val_loss: 0.3057 - val_acc: 0.8692
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 1s - loss: 0.1196 - acc: 0.9500 - val_loss: 0.3172 - val_acc: 0.8879
Epoch 13

Epoch 41/150
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.2138 - acc: 0.9078 - val_loss: 0.2514 - val_acc: 0.8879
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.1980 - acc: 0.9172 - val_loss: 0.2546 - val_acc: 0.8972
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.1895 - acc: 0.9258 - val_loss: 0.2526 - val_acc: 0.8972
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.1904 - acc: 0.9226 - val_loss: 0.2642 - val_acc: 0.8941
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.2132 - acc: 0.9141 - val_loss: 0.2508 - val_acc: 0.8910
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 1s - loss: 0.1791 - acc: 0.9226 - val_loss: 0.2568 - val_acc: 0.8879
Epoch 47/150
Epoch 00047: val_loss improved from 0.24726 to 0.24592, saving model to best_m.h5
 - 1s - loss: 0.1947 - acc: 0.9164 - val_loss: 0.2459 - val_acc: 0.9034
Epoch 48/150
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.1937 - acc: 0.9117 - val_los

Epoch 107/150
Epoch 00107: val_loss did not improve
 - 1s - loss: 0.1457 - acc: 0.9398 - val_loss: 0.2766 - val_acc: 0.9034
Epoch 108/150
Epoch 00108: val_loss did not improve
 - 2s - loss: 0.1502 - acc: 0.9359 - val_loss: 0.2821 - val_acc: 0.9159
Epoch 109/150
Epoch 00109: val_loss did not improve
 - 1s - loss: 0.1209 - acc: 0.9492 - val_loss: 0.2880 - val_acc: 0.9190
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 1s - loss: 0.1567 - acc: 0.9240 - val_loss: 0.2726 - val_acc: 0.9065
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 1s - loss: 0.1401 - acc: 0.9359 - val_loss: 0.2880 - val_acc: 0.8879
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 1s - loss: 0.1368 - acc: 0.9422 - val_loss: 0.2895 - val_acc: 0.9003
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 1s - loss: 0.1417 - acc: 0.9430 - val_loss: 0.2875 - val_acc: 0.9065
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 1s - loss: 0.1523 - acc: 0.9391 - val_loss: 0.2764 - val_acc: 0.8972
Epoch 11

...

Epoch 2/150
Epoch 00002: val_loss improved from 0.58986 to 0.46773, saving model to best_m.h5
 - 1s - loss: 0.5705 - acc: 0.7445 - val_loss: 0.4677 - val_acc: 0.8131
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 1s - loss: 0.4916 - acc: 0.7896 - val_loss: 0.5071 - val_acc: 0.7726
Epoch 4/150
Epoch 00004: val_loss improved from 0.46773 to 0.33996, saving model to best_m.h5
 - 1s - loss: 0.4184 - acc: 0.8187 - val_loss: 0.3400 - val_acc: 0.8536
Epoch 5/150
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.4310 - acc: 0.7974 - val_loss: 0.3470 - val_acc: 0.8629
Epoch 6/150
Epoch 00006: val_loss improved from 0.33996 to 0.31867, saving model to best_m.h5
 - 1s - loss: 0.3976 - acc: 0.8240 - val_loss: 0.3187 - val_acc: 0.8816
Epoch 7/150
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.3561 - acc: 0.8594 - val_loss: 0.3251 - val_acc: 0.8505
Epoch 8/150
Epoch 00008: val_loss improved from 0.31867 to 0.31762, saving model to best_m.h5
 - 1s - loss: 0.3422 - acc: 0.8558 - val_l

Epoch 63/150
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1754 - acc: 0.9266 - val_loss: 0.2609 - val_acc: 0.8847
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1731 - acc: 0.9226 - val_loss: 0.2635 - val_acc: 0.8910
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1794 - acc: 0.9172 - val_loss: 0.2616 - val_acc: 0.8941
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1617 - acc: 0.9281 - val_loss: 0.2718 - val_acc: 0.8910
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1789 - acc: 0.9138 - val_loss: 0.2791 - val_acc: 0.8847
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1693 - acc: 0.9265 - val_loss: 0.2614 - val_acc: 0.8972
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1677 - acc: 0.9273 - val_loss: 0.2822 - val_acc: 0.9003
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1644 - acc: 0.9312 - val_loss: 0.2551 - val_acc: 0.9097
Epoch 71/150
Epo

Epoch 00129: val_loss did not improve
 - 1s - loss: 0.1239 - acc: 0.9490 - val_loss: 0.2793 - val_acc: 0.8785
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 1s - loss: 0.1232 - acc: 0.9506 - val_loss: 0.2936 - val_acc: 0.8847
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 1s - loss: 0.1243 - acc: 0.9414 - val_loss: 0.2602 - val_acc: 0.9003
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 2s - loss: 0.1302 - acc: 0.9453 - val_loss: 0.2804 - val_acc: 0.8910
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 1s - loss: 0.1165 - acc: 0.9539 - val_loss: 0.2943 - val_acc: 0.8910
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 1s - loss: 0.1152 - acc: 0.9506 - val_loss: 0.3094 - val_acc: 0.8972
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 2s - loss: 0.1147 - acc: 0.9523 - val_loss: 0.3182 - val_acc: 0.8847
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 1s - loss: 0.1031 - acc: 0.9547 - val_loss: 0.3102 - val_acc: 0.8785
Epoch 137/150
Epoch 00

Epoch 43/150
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.1676 - acc: 0.9320 - val_loss: 0.3204 - val_acc: 0.8875
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1889 - acc: 0.9141 - val_loss: 0.3132 - val_acc: 0.8781
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.1816 - acc: 0.9297 - val_loss: 0.3142 - val_acc: 0.8875
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 1s - loss: 0.1636 - acc: 0.9297 - val_loss: 0.3311 - val_acc: 0.8719
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.1807 - acc: 0.9195 - val_loss: 0.3334 - val_acc: 0.8688
Epoch 48/150
Epoch 00048: val_loss did not improve
 - 1s - loss: 0.2027 - acc: 0.9164 - val_loss: 0.3085 - val_acc: 0.8594
Epoch 49/150
Epoch 00049: val_loss did not improve
 - 1s - loss: 0.1819 - acc: 0.9125 - val_loss: 0.3211 - val_acc: 0.8781
Epoch 50/150
Epoch 00050: val_loss did not improve
 - 1s - loss: 0.1905 - acc: 0.9156 - val_loss: 0.3320 - val_acc: 0.8781
Epoch 51/150
Epo

Epoch 110/150
Epoch 00110: val_loss did not improve
 - 1s - loss: 0.1398 - acc: 0.9406 - val_loss: 0.3674 - val_acc: 0.8594
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 1s - loss: 0.1404 - acc: 0.9383 - val_loss: 0.3505 - val_acc: 0.8594
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 1s - loss: 0.1272 - acc: 0.9438 - val_loss: 0.3693 - val_acc: 0.8688
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 1s - loss: 0.1316 - acc: 0.9492 - val_loss: 0.3541 - val_acc: 0.8688
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 1s - loss: 0.1276 - acc: 0.9539 - val_loss: 0.3577 - val_acc: 0.8938
Epoch 115/150
Epoch 00115: val_loss did not improve
 - 1s - loss: 0.1247 - acc: 0.9508 - val_loss: 0.3897 - val_acc: 0.8844
Epoch 116/150
Epoch 00116: val_loss did not improve
 - 1s - loss: 0.1388 - acc: 0.9414 - val_loss: 0.3559 - val_acc: 0.8750
Epoch 117/150
Epoch 00117: val_loss did not improve
 - 1s - loss: 0.1184 - acc: 0.9539 - val_loss: 0.3578 - val_acc: 0.8688
Epoch 11

In [10]:
with open('../features/cnn_2_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2728
# skimage 2710

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug_denoise_preprocess.csv', index=False)

0.238003406571
         id  is_iceberg
0  5941774d    0.117950
1  4023181e    0.485464
2  b20200e4    0.152887
3  e7f018bb    0.995076
4  4371c8c3    0.000353


In [11]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [12]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.59938, saving model to best_m.h5
 - 4s - loss: 0.6780 - acc: 0.5823 - val_loss: 0.5994 - val_acc: 0.6604
Epoch 2/150
Epoch 00002: val_loss improved from 0.59938 to 0.54705, saving model to best_m.h5
 - 2s - loss: 0.5930 - acc: 0.6872 - val_loss: 0.5470 - val_acc: 0.7882
Epoch 3/150
Epoch 00003: val_loss improved from 0.54705 to 0.44937, saving model to best_m.h5
 - 2s - loss: 0.5331 - acc: 0.7448 - val_loss: 0.4494 - val_acc: 0.8318
Epoch 4/150
Epoch 00004: val_loss improved from 0.44937 to 0.39390, saving model to best_m.h5
 - 2s - loss: 0.4467 - acc: 0.8039 - val_loss: 0.3939 - val_acc: 0.8318
Epoch 5/150
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.4195 - acc: 0.8130 - val_loss: 0.3962 - val_acc: 0.8162
Epoch 6/150
Epoch 00006: val_loss improved from 0.39390 to 0.37563, saving model to best_m.h5
 - 2s - loss: 0.4311 - acc: 0.8021 - val_loss: 0.3756 - val_acc: 0.8224
Epoch 7/150
Epoch 00007: val_loss improved from 0.37

Epoch 62/150
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.2434 - acc: 0.8958 - val_loss: 0.2578 - val_acc: 0.8785
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2272 - acc: 0.9045 - val_loss: 0.2553 - val_acc: 0.8816
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2153 - acc: 0.9039 - val_loss: 0.2496 - val_acc: 0.8910
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.2432 - acc: 0.8974 - val_loss: 0.2556 - val_acc: 0.8847
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.2106 - acc: 0.9164 - val_loss: 0.2490 - val_acc: 0.8879
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2136 - acc: 0.9109 - val_loss: 0.2516 - val_acc: 0.8941
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2151 - acc: 0.9076 - val_loss: 0.2543 - val_acc: 0.8879
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2180 - acc: 0.9011 - val_loss: 0.2515 - val_acc: 0.8816
Epoch 70/150
Epo

Epoch 00128: val_loss did not improve
 - 2s - loss: 0.1797 - acc: 0.9242 - val_loss: 0.2523 - val_acc: 0.8972
Epoch 129/150
Epoch 00129: val_loss did not improve
 - 2s - loss: 0.1980 - acc: 0.9133 - val_loss: 0.2603 - val_acc: 0.8910
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 2s - loss: 0.1718 - acc: 0.9297 - val_loss: 0.2465 - val_acc: 0.8910
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 2s - loss: 0.1906 - acc: 0.9164 - val_loss: 0.2717 - val_acc: 0.8816
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 2s - loss: 0.1808 - acc: 0.9195 - val_loss: 0.2672 - val_acc: 0.8754
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 2s - loss: 0.1814 - acc: 0.9224 - val_loss: 0.2627 - val_acc: 0.8754
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 2s - loss: 0.1715 - acc: 0.9320 - val_loss: 0.2594 - val_acc: 0.8879
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 2s - loss: 0.1884 - acc: 0.9209 - val_loss: 0.2434 - val_acc: 0.8910
Epoch 136/150
Epoch 00

Epoch 39/150
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.2173 - acc: 0.9156 - val_loss: 0.2843 - val_acc: 0.8785
Epoch 40/150
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.2268 - acc: 0.8920 - val_loss: 0.2832 - val_acc: 0.8941
Epoch 41/150
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.2185 - acc: 0.9068 - val_loss: 0.2763 - val_acc: 0.8972
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.2288 - acc: 0.9000 - val_loss: 0.2912 - val_acc: 0.8847
Epoch 43/150
Epoch 00043: val_loss improved from 0.27367 to 0.26659, saving model to best_m.h5
 - 2s - loss: 0.2167 - acc: 0.9070 - val_loss: 0.2666 - val_acc: 0.8972
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.2169 - acc: 0.9094 - val_loss: 0.2734 - val_acc: 0.8816
Epoch 45/150
Epoch 00045: val_loss improved from 0.26659 to 0.26324, saving model to best_m.h5
 - 2s - loss: 0.2398 - acc: 0.8961 - val_loss: 0.2632 - val_acc: 0.8972
Epoch 46/150
Epoch 00046: val_loss did not improve


Epoch 104/150
Epoch 00104: val_loss did not improve
 - 2s - loss: 0.1883 - acc: 0.9156 - val_loss: 0.2693 - val_acc: 0.9003
Epoch 105/150
Epoch 00105: val_loss did not improve
 - 2s - loss: 0.1696 - acc: 0.9250 - val_loss: 0.2610 - val_acc: 0.9065
Epoch 106/150
Epoch 00106: val_loss did not improve
 - 2s - loss: 0.1908 - acc: 0.9203 - val_loss: 0.2598 - val_acc: 0.8941
Epoch 107/150
Epoch 00107: val_loss did not improve
 - 2s - loss: 0.1981 - acc: 0.9115 - val_loss: 0.2670 - val_acc: 0.8972
Epoch 108/150
Epoch 00108: val_loss did not improve
 - 2s - loss: 0.2013 - acc: 0.9172 - val_loss: 0.2620 - val_acc: 0.8941
Epoch 109/150
Epoch 00109: val_loss did not improve
 - 2s - loss: 0.1755 - acc: 0.9265 - val_loss: 0.2685 - val_acc: 0.8972
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 2s - loss: 0.1852 - acc: 0.9219 - val_loss: 0.2927 - val_acc: 0.8910
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 2s - loss: 0.1833 - acc: 0.9156 - val_loss: 0.2658 - val_acc: 0.8941
Epoch 11

...

Epoch 147/150
Epoch 00147: val_loss did not improve
 - 2s - loss: 0.1762 - acc: 0.9289 - val_loss: 0.2446 - val_acc: 0.9003
Epoch 148/150
Epoch 00148: val_loss did not improve
 - 2s - loss: 0.1651 - acc: 0.9305 - val_loss: 0.2295 - val_acc: 0.8972
Epoch 149/150
Epoch 00149: val_loss did not improve
 - 2s - loss: 0.1697 - acc: 0.9226 - val_loss: 0.2312 - val_acc: 0.9065
Epoch 150/150
Epoch 00150: val_loss did not improve
 - 2s - loss: 0.1736 - acc: 0.9266 - val_loss: 0.2366 - val_acc: 0.9128
Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.55878, saving model to best_m.h5
 - 4s - loss: 0.6751 - acc: 0.5786 - val_loss: 0.5588 - val_acc: 0.7227
Epoch 2/150
Epoch 00002: val_loss improved from 0.55878 to 0.49111, saving model to best_m.h5
 - 2s - loss: 0.5531 - acc: 0.7125 - val_loss: 0.4911 - val_acc: 0.7664
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4841 - acc: 0.7902 - val_loss: 0.5207 - val_acc: 0.7757
Epoch 4/150
Epoch 00004: val_loss improved from 0.4911

Epoch 59/150
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.2161 - acc: 0.9062 - val_loss: 0.2585 - val_acc: 0.8941
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 2s - loss: 0.2191 - acc: 0.9094 - val_loss: 0.2480 - val_acc: 0.9003
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.2170 - acc: 0.9078 - val_loss: 0.2516 - val_acc: 0.8972
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.2231 - acc: 0.9055 - val_loss: 0.2448 - val_acc: 0.8941
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2141 - acc: 0.9091 - val_loss: 0.2655 - val_acc: 0.8941
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2246 - acc: 0.9003 - val_loss: 0.2508 - val_acc: 0.8879
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.2035 - acc: 0.9148 - val_loss: 0.2488 - val_acc: 0.8879
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.2180 - acc: 0.9109 - val_loss: 0.3063 - val_acc: 0.8816
Epoch 67/150
Epo

Epoch 00125: val_loss did not improve
 - 2s - loss: 0.1895 - acc: 0.9234 - val_loss: 0.2618 - val_acc: 0.8972
Epoch 126/150
Epoch 00126: val_loss did not improve
 - 2s - loss: 0.1931 - acc: 0.9164 - val_loss: 0.2737 - val_acc: 0.8910
Epoch 127/150
Epoch 00127: val_loss did not improve
 - 2s - loss: 0.1814 - acc: 0.9187 - val_loss: 0.2633 - val_acc: 0.9034
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 2s - loss: 0.1680 - acc: 0.9344 - val_loss: 0.2529 - val_acc: 0.9065
Epoch 129/150
Epoch 00129: val_loss did not improve
 - 2s - loss: 0.1979 - acc: 0.9164 - val_loss: 0.2528 - val_acc: 0.8941
Epoch 130/150
Epoch 00130: val_loss did not improve
 - 2s - loss: 0.1696 - acc: 0.9328 - val_loss: 0.2626 - val_acc: 0.8941
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 2s - loss: 0.1961 - acc: 0.9170 - val_loss: 0.2700 - val_acc: 0.8879
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 2s - loss: 0.1899 - acc: 0.9266 - val_loss: 0.2611 - val_acc: 0.8847
Epoch 133/150
Epoch 00

Epoch 37/150
Epoch 00037: val_loss did not improve
 - 2s - loss: 0.2112 - acc: 0.8969 - val_loss: 0.2928 - val_acc: 0.8906
Epoch 38/150
Epoch 00038: val_loss did not improve
 - 2s - loss: 0.2044 - acc: 0.9172 - val_loss: 0.2893 - val_acc: 0.8938
Epoch 39/150
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.2117 - acc: 0.9070 - val_loss: 0.3000 - val_acc: 0.8875
Epoch 40/150
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.2233 - acc: 0.8969 - val_loss: 0.2923 - val_acc: 0.8875
Epoch 41/150
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.2068 - acc: 0.9211 - val_loss: 0.2896 - val_acc: 0.8844
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.2428 - acc: 0.8930 - val_loss: 0.2836 - val_acc: 0.8969
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.2243 - acc: 0.8937 - val_loss: 0.2860 - val_acc: 0.8875
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1950 - acc: 0.9156 - val_loss: 0.3011 - val_acc: 0.8844
Epoch 45/150
Epo

Epoch 103/150
Epoch 00103: val_loss did not improve
 - 2s - loss: 0.1673 - acc: 0.9313 - val_loss: 0.2948 - val_acc: 0.8906
Epoch 104/150
Epoch 00104: val_loss did not improve
 - 2s - loss: 0.1869 - acc: 0.9118 - val_loss: 0.2956 - val_acc: 0.8938
Epoch 105/150
Epoch 00105: val_loss did not improve
 - 2s - loss: 0.1683 - acc: 0.9398 - val_loss: 0.3093 - val_acc: 0.8906
Epoch 106/150
Epoch 00106: val_loss did not improve
 - 2s - loss: 0.1859 - acc: 0.9188 - val_loss: 0.2866 - val_acc: 0.9000
Epoch 107/150
Epoch 00107: val_loss did not improve
 - 2s - loss: 0.1670 - acc: 0.9149 - val_loss: 0.2900 - val_acc: 0.8969
Epoch 108/150
Epoch 00108: val_loss did not improve
 - 2s - loss: 0.1542 - acc: 0.9336 - val_loss: 0.3003 - val_acc: 0.8938
Epoch 109/150
Epoch 00109: val_loss did not improve
 - 2s - loss: 0.1569 - acc: 0.9289 - val_loss: 0.2924 - val_acc: 0.8969
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 2s - loss: 0.1562 - acc: 0.9422 - val_loss: 0.3049 - val_acc: 0.8875
Epoch 11

In [13]:
with open('../features/cnn_3_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2737
# new  2509

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug_denoise_preprocess.csv', index=False)


0.228912552147
         id    is_iceberg
0  5941774d  2.635534e-01
1  4023181e  2.095651e-01
2  b20200e4  2.171637e-02
3  e7f018bb  9.969970e-01
4  4371c8c3  3.532075e-07


In [14]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [15]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.52968, saving model to best_m.h5
 - 4s - loss: 0.7071 - acc: 0.7179 - val_loss: 0.5297 - val_acc: 0.7477
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.4263 - acc: 0.8101 - val_loss: 0.5807 - val_acc: 0.6262
Epoch 3/150
Epoch 00003: val_loss improved from 0.52968 to 0.46364, saving model to best_m.h5
 - 3s - loss: 0.4264 - acc: 0.8216 - val_loss: 0.4636 - val_acc: 0.7695
Epoch 4/150
Epoch 00004: val_loss improved from 0.46364 to 0.45002, saving model to best_m.h5
 - 2s - loss: 0.3680 - acc: 0.8435 - val_loss: 0.4500 - val_acc: 0.7975
Epoch 5/150
Epoch 00005: val_loss improved from 0.45002 to 0.44452, saving model to best_m.h5
 - 3s - loss: 0.3877 - acc: 0.8326 - val_loss: 0.4445 - val_acc: 0.8131
Epoch 6/150
Epoch 00006: val_loss improved from 0.44452 to 0.40497, saving model to best_m.h5
 - 2s - loss: 0.3629 - acc: 0.8341 - val_loss: 0.4050 - val_acc: 0.8442
Epoch 7/150
Epoch 00007: val_loss did not improve
 -

Epoch 64/150
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1307 - acc: 0.9477 - val_loss: 0.2733 - val_acc: 0.8847
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.1102 - acc: 0.9578 - val_loss: 0.2782 - val_acc: 0.8816
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1413 - acc: 0.9437 - val_loss: 0.2760 - val_acc: 0.8754
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.1047 - acc: 0.9617 - val_loss: 0.2643 - val_acc: 0.8692
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1232 - acc: 0.9562 - val_loss: 0.3077 - val_acc: 0.8723
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.1057 - acc: 0.9586 - val_loss: 0.2935 - val_acc: 0.8692
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1101 - acc: 0.9578 - val_loss: 0.3177 - val_acc: 0.8660
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 3s - loss: 0.1238 - acc: 0.9469 - val_loss: 0.3186 - val_acc: 0.8692
Epoch 72/150
Epo

Epoch 131/150
Epoch 00131: val_loss did not improve
 - 3s - loss: 0.0499 - acc: 0.9820 - val_loss: 0.3690 - val_acc: 0.8754
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 2s - loss: 0.0635 - acc: 0.9797 - val_loss: 0.3869 - val_acc: 0.8785
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 3s - loss: 0.0751 - acc: 0.9685 - val_loss: 0.3890 - val_acc: 0.8754
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 2s - loss: 0.0715 - acc: 0.9750 - val_loss: 0.3878 - val_acc: 0.8536
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 2s - loss: 0.0586 - acc: 0.9789 - val_loss: 0.3878 - val_acc: 0.8785
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 2s - loss: 0.0444 - acc: 0.9852 - val_loss: 0.3844 - val_acc: 0.8442
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 2s - loss: 0.0512 - acc: 0.9844 - val_loss: 0.3831 - val_acc: 0.8598
Epoch 138/150
Epoch 00138: val_loss did not improve
 - 2s - loss: 0.0504 - acc: 0.9844 - val_loss: 0.3936 - val_acc: 0.8567
Epoch 13

Epoch 44/150
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1749 - acc: 0.9232 - val_loss: 0.2760 - val_acc: 0.8879
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 3s - loss: 0.1525 - acc: 0.9313 - val_loss: 0.2753 - val_acc: 0.8879
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.1435 - acc: 0.9427 - val_loss: 0.2766 - val_acc: 0.8972
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 3s - loss: 0.1528 - acc: 0.9367 - val_loss: 0.2717 - val_acc: 0.8941
Epoch 48/150
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.1507 - acc: 0.9351 - val_loss: 0.3054 - val_acc: 0.8879
Epoch 49/150
Epoch 00049: val_loss did not improve
 - 3s - loss: 0.1558 - acc: 0.9414 - val_loss: 0.2795 - val_acc: 0.9003
Epoch 50/150
Epoch 00050: val_loss did not improve
 - 2s - loss: 0.1458 - acc: 0.9404 - val_loss: 0.2831 - val_acc: 0.8941
Epoch 51/150
Epoch 00051: val_loss did not improve
 - 3s - loss: 0.1582 - acc: 0.9406 - val_loss: 0.2831 - val_acc: 0.8941
Epoch 52/150
Epo

Epoch 111/150
Epoch 00111: val_loss did not improve
 - 2s - loss: 0.0906 - acc: 0.9625 - val_loss: 0.3113 - val_acc: 0.9003
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 2s - loss: 0.0887 - acc: 0.9711 - val_loss: 0.3052 - val_acc: 0.8785
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 2s - loss: 0.0718 - acc: 0.9703 - val_loss: 0.3207 - val_acc: 0.8816
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 2s - loss: 0.0714 - acc: 0.9758 - val_loss: 0.3112 - val_acc: 0.8972
Epoch 115/150
Epoch 00115: val_loss did not improve
 - 2s - loss: 0.0703 - acc: 0.9750 - val_loss: 0.3241 - val_acc: 0.8910
Epoch 116/150
Epoch 00116: val_loss did not improve
 - 2s - loss: 0.0653 - acc: 0.9812 - val_loss: 0.3480 - val_acc: 0.8941
Epoch 117/150
Epoch 00117: val_loss did not improve
 - 2s - loss: 0.0559 - acc: 0.9859 - val_loss: 0.3310 - val_acc: 0.8941
Epoch 118/150
Epoch 00118: val_loss did not improve
 - 2s - loss: 0.0993 - acc: 0.9654 - val_loss: 0.3489 - val_acc: 0.8879
Epoch 11

...

Epoch 7/150
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.2942 - acc: 0.8734 - val_loss: 0.3239 - val_acc: 0.8349
Epoch 8/150
Epoch 00008: val_loss improved from 0.31804 to 0.26046, saving model to best_m.h5
 - 2s - loss: 0.3328 - acc: 0.8555 - val_loss: 0.2605 - val_acc: 0.8785
Epoch 9/150
Epoch 00009: val_loss did not improve
 - 2s - loss: 0.3451 - acc: 0.8523 - val_loss: 0.2724 - val_acc: 0.8847
Epoch 10/150
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.3063 - acc: 0.8656 - val_loss: 0.3088 - val_acc: 0.8567
Epoch 11/150
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.2653 - acc: 0.8945 - val_loss: 0.3179 - val_acc: 0.8536
Epoch 12/150
Epoch 00012: val_loss improved from 0.26046 to 0.23914, saving model to best_m.h5
 - 2s - loss: 0.2472 - acc: 0.8966 - val_loss: 0.2391 - val_acc: 0.8816
Epoch 13/150
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2636 - acc: 0.8935 - val_loss: 0.2809 - val_acc: 0.8785
Epoch 14/150
Epoch 00014: val_loss did not improve
 - 

Epoch 73/150
Epoch 00073: val_loss did not improve
 - 3s - loss: 0.1092 - acc: 0.9594 - val_loss: 0.2705 - val_acc: 0.9065
Epoch 74/150
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1084 - acc: 0.9617 - val_loss: 0.3124 - val_acc: 0.8972
Epoch 75/150
Epoch 00075: val_loss did not improve
 - 3s - loss: 0.0996 - acc: 0.9633 - val_loss: 0.2615 - val_acc: 0.9065
Epoch 76/150
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.1015 - acc: 0.9584 - val_loss: 0.2640 - val_acc: 0.9034
Epoch 77/150
Epoch 00077: val_loss did not improve
 - 3s - loss: 0.1099 - acc: 0.9586 - val_loss: 0.2618 - val_acc: 0.9034
Epoch 78/150
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.0913 - acc: 0.9680 - val_loss: 0.2831 - val_acc: 0.9034
Epoch 79/150
Epoch 00079: val_loss did not improve
 - 3s - loss: 0.0775 - acc: 0.9727 - val_loss: 0.2879 - val_acc: 0.8941
Epoch 80/150
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1088 - acc: 0.9602 - val_loss: 0.2598 - val_acc: 0.9003
Epoch 81/150
Epo

Epoch 140/150
Epoch 00140: val_loss did not improve
 - 3s - loss: 0.0467 - acc: 0.9859 - val_loss: 0.3579 - val_acc: 0.8785
Epoch 141/150
Epoch 00141: val_loss did not improve
 - 2s - loss: 0.0542 - acc: 0.9805 - val_loss: 0.3378 - val_acc: 0.9003
Epoch 142/150
Epoch 00142: val_loss did not improve
 - 3s - loss: 0.0398 - acc: 0.9898 - val_loss: 0.3746 - val_acc: 0.8816
Epoch 143/150
Epoch 00143: val_loss did not improve
 - 2s - loss: 0.0382 - acc: 0.9859 - val_loss: 0.3490 - val_acc: 0.8972
Epoch 144/150
Epoch 00144: val_loss did not improve
 - 3s - loss: 0.0496 - acc: 0.9852 - val_loss: 0.3375 - val_acc: 0.8847
Epoch 145/150
Epoch 00145: val_loss did not improve
 - 2s - loss: 0.0480 - acc: 0.9859 - val_loss: 0.3588 - val_acc: 0.8910
Epoch 146/150
Epoch 00146: val_loss did not improve
 - 3s - loss: 0.0522 - acc: 0.9859 - val_loss: 0.3511 - val_acc: 0.8754
Epoch 147/150
Epoch 00147: val_loss did not improve
 - 2s - loss: 0.0527 - acc: 0.9812 - val_loss: 0.3716 - val_acc: 0.8847
Epoch 14

Epoch 54/150
Epoch 00054: val_loss did not improve
 - 2s - loss: 0.1089 - acc: 0.9602 - val_loss: 0.3652 - val_acc: 0.8906
Epoch 55/150
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.1323 - acc: 0.9516 - val_loss: 0.3742 - val_acc: 0.8750
Epoch 56/150
Epoch 00056: val_loss did not improve
 - 3s - loss: 0.1216 - acc: 0.9516 - val_loss: 0.3844 - val_acc: 0.8844
Epoch 57/150
Epoch 00057: val_loss did not improve
 - 2s - loss: 0.1364 - acc: 0.9469 - val_loss: 0.3990 - val_acc: 0.8531
Epoch 58/150
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.1171 - acc: 0.9547 - val_loss: 0.3800 - val_acc: 0.8719
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.1169 - acc: 0.9539 - val_loss: 0.3915 - val_acc: 0.8469
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 2s - loss: 0.1105 - acc: 0.9602 - val_loss: 0.3598 - val_acc: 0.8594
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.1092 - acc: 0.9594 - val_loss: 0.3770 - val_acc: 0.8688
Epoch 62/150
Epo

Epoch 121/150
Epoch 00121: val_loss did not improve
 - 3s - loss: 0.0606 - acc: 0.9813 - val_loss: 0.4808 - val_acc: 0.8906
Epoch 122/150
Epoch 00122: val_loss did not improve
 - 2s - loss: 0.0557 - acc: 0.9828 - val_loss: 0.4557 - val_acc: 0.8625
Epoch 123/150
Epoch 00123: val_loss did not improve
 - 3s - loss: 0.0729 - acc: 0.9695 - val_loss: 0.4452 - val_acc: 0.8656
Epoch 124/150
Epoch 00124: val_loss did not improve
 - 3s - loss: 0.0527 - acc: 0.9836 - val_loss: 0.5180 - val_acc: 0.8750
Epoch 125/150
Epoch 00125: val_loss did not improve
 - 3s - loss: 0.0559 - acc: 0.9821 - val_loss: 0.4612 - val_acc: 0.8938
Epoch 126/150
Epoch 00126: val_loss did not improve
 - 3s - loss: 0.0623 - acc: 0.9766 - val_loss: 0.5095 - val_acc: 0.8906
Epoch 127/150
Epoch 00127: val_loss did not improve
 - 2s - loss: 0.0380 - acc: 0.9859 - val_loss: 0.4618 - val_acc: 0.8750
Epoch 128/150
Epoch 00128: val_loss did not improve
 - 2s - loss: 0.0558 - acc: 0.9805 - val_loss: 0.4668 - val_acc: 0.8719
Epoch 12

In [16]:
with open('../features/cnn_4_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2146

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug_denoise_preprocess.csv', index=False)

0.238796796671
         id  is_iceberg
0  5941774d    0.075453
1  4023181e    0.395174
2  b20200e4    0.086798
3  e7f018bb    0.983354
4  4371c8c3    0.001708


In [17]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [18]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=150, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.63615, saving model to best_m.h5
 - 7s - loss: 0.6635 - acc: 0.6359 - val_loss: 0.6361 - val_acc: 0.6417
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.5329 - acc: 0.7495 - val_loss: 0.6833 - val_acc: 0.5701
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4467 - acc: 0.7843 - val_loss: 0.7905 - val_acc: 0.6386
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.4382 - acc: 0.7966 - val_loss: 0.6389 - val_acc: 0.6822
Epoch 5/150
Epoch 00005: val_loss improved from 0.63615 to 0.52713, saving model to best_m.h5
 - 4s - loss: 0.3927 - acc: 0.8206 - val_loss: 0.5271 - val_acc: 0.7788
Epoch 6/150
Epoch 00006: val_loss improved from 0.52713 to 0.44440, saving model to best_m.h5
 - 4s - loss: 0.4292 - acc: 0.8027 - val_loss: 0.4444 - val_acc: 0.7788
Epoch 7/150
Epoch 00007: val_loss improved from 0.44440 to 0.38820, saving model to best_m.h5
 - 3s - loss: 0.3892 - acc: 0.8154 - val_loss:

Epoch 64/150
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.2507 - acc: 0.8937 - val_loss: 0.2385 - val_acc: 0.9034
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.2319 - acc: 0.9101 - val_loss: 0.2319 - val_acc: 0.8910
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.2580 - acc: 0.8898 - val_loss: 0.2410 - val_acc: 0.9034
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.2438 - acc: 0.8982 - val_loss: 0.2375 - val_acc: 0.8972
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.2275 - acc: 0.9052 - val_loss: 0.2303 - val_acc: 0.8941
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 4s - loss: 0.2364 - acc: 0.9031 - val_loss: 0.2302 - val_acc: 0.9159
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.2407 - acc: 0.8922 - val_loss: 0.2473 - val_acc: 0.8972
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 4s - loss: 0.2279 - acc: 0.8992 - val_loss: 0.2457 - val_acc: 0.8941
Epoch 72/150
Epo

Epoch 131/150
Epoch 00131: val_loss did not improve
 - 4s - loss: 0.2079 - acc: 0.9070 - val_loss: 0.2558 - val_acc: 0.9003
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 4s - loss: 0.1857 - acc: 0.9216 - val_loss: 0.2593 - val_acc: 0.9034
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 3s - loss: 0.2128 - acc: 0.8980 - val_loss: 0.2702 - val_acc: 0.8972
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 4s - loss: 0.1957 - acc: 0.9094 - val_loss: 0.2430 - val_acc: 0.8972
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 4s - loss: 0.1981 - acc: 0.9156 - val_loss: 0.2496 - val_acc: 0.9034
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 4s - loss: 0.1984 - acc: 0.9195 - val_loss: 0.2619 - val_acc: 0.8879
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 4s - loss: 0.2152 - acc: 0.9148 - val_loss: 0.2552 - val_acc: 0.9003
Epoch 138/150
Epoch 00138: val_loss did not improve
 - 4s - loss: 0.2462 - acc: 0.8920 - val_loss: 0.2839 - val_acc: 0.8910
Epoch 13

Epoch 44/150
Epoch 00044: val_loss did not improve
 - 4s - loss: 0.2587 - acc: 0.9008 - val_loss: 0.2973 - val_acc: 0.8629
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 4s - loss: 0.2519 - acc: 0.8935 - val_loss: 0.2990 - val_acc: 0.8754
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 4s - loss: 0.2401 - acc: 0.8945 - val_loss: 0.2952 - val_acc: 0.8785
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 4s - loss: 0.2603 - acc: 0.8995 - val_loss: 0.2994 - val_acc: 0.8660
Epoch 48/150
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.2629 - acc: 0.8906 - val_loss: 0.2957 - val_acc: 0.8785
Epoch 49/150
Epoch 00049: val_loss did not improve
 - 4s - loss: 0.2460 - acc: 0.8961 - val_loss: 0.2893 - val_acc: 0.8879
Epoch 50/150
Epoch 00050: val_loss did not improve
 - 4s - loss: 0.2481 - acc: 0.8990 - val_loss: 0.2807 - val_acc: 0.8754
Epoch 51/150
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.2364 - acc: 0.9047 - val_loss: 0.3015 - val_acc: 0.8785
Epoch 52/150
Epo

Epoch 109/150
Epoch 00109: val_loss did not improve
 - 4s - loss: 0.2064 - acc: 0.9109 - val_loss: 0.2995 - val_acc: 0.8879
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 4s - loss: 0.2021 - acc: 0.9226 - val_loss: 0.2766 - val_acc: 0.8941
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 4s - loss: 0.1909 - acc: 0.9172 - val_loss: 0.2847 - val_acc: 0.8972
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 4s - loss: 0.2048 - acc: 0.9099 - val_loss: 0.2875 - val_acc: 0.8816
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 4s - loss: 0.1982 - acc: 0.9180 - val_loss: 0.2730 - val_acc: 0.8847
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 4s - loss: 0.1894 - acc: 0.9109 - val_loss: 0.2990 - val_acc: 0.9003
Epoch 115/150
Epoch 00115: val_loss did not improve
 - 4s - loss: 0.2082 - acc: 0.9140 - val_loss: 0.2827 - val_acc: 0.8972
Epoch 116/150
Epoch 00116: val_loss did not improve
 - 4s - loss: 0.2059 - acc: 0.9133 - val_loss: 0.2746 - val_acc: 0.8785
Epoch 11

...

Epoch 00001: val_loss improved from inf to 0.78631, saving model to best_m.h5
 - 8s - loss: 0.6422 - acc: 0.6440 - val_loss: 0.7863 - val_acc: 0.4766
Epoch 2/150
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.5248 - acc: 0.7578 - val_loss: 0.8611 - val_acc: 0.5514
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.4774 - acc: 0.7724 - val_loss: 0.8469 - val_acc: 0.7227
Epoch 4/150
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.4280 - acc: 0.7859 - val_loss: 0.8914 - val_acc: 0.7477
Epoch 5/150
Epoch 00005: val_loss improved from 0.78631 to 0.38683, saving model to best_m.h5
 - 4s - loss: 0.4170 - acc: 0.8094 - val_loss: 0.3868 - val_acc: 0.8287
Epoch 6/150
Epoch 00006: val_loss improved from 0.38683 to 0.36087, saving model to best_m.h5
 - 4s - loss: 0.4011 - acc: 0.8234 - val_loss: 0.3609 - val_acc: 0.8255
Epoch 7/150
Epoch 00007: val_loss improved from 0.36087 to 0.35983, saving model to best_m.h5
 - 4s - loss: 0.3654 - acc: 0.8461 - val_loss: 0.3598 - va

Epoch 64/150
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.2493 - acc: 0.8867 - val_loss: 0.2608 - val_acc: 0.9003
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.2388 - acc: 0.8984 - val_loss: 0.2708 - val_acc: 0.9034
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.2632 - acc: 0.8984 - val_loss: 0.2723 - val_acc: 0.8972
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.2190 - acc: 0.9031 - val_loss: 0.3084 - val_acc: 0.8972
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.2727 - acc: 0.8792 - val_loss: 0.2877 - val_acc: 0.9003
Epoch 69/150
Epoch 00069: val_loss did not improve
 - 4s - loss: 0.2504 - acc: 0.8984 - val_loss: 0.2715 - val_acc: 0.8941
Epoch 70/150
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.2343 - acc: 0.9016 - val_loss: 0.2659 - val_acc: 0.8910
Epoch 71/150
Epoch 00071: val_loss did not improve
 - 4s - loss: 0.2442 - acc: 0.9016 - val_loss: 0.2564 - val_acc: 0.8972
Epoch 72/150
Epo

Epoch 130/150
Epoch 00130: val_loss did not improve
 - 4s - loss: 0.2060 - acc: 0.9156 - val_loss: 0.2955 - val_acc: 0.8816
Epoch 131/150
Epoch 00131: val_loss did not improve
 - 4s - loss: 0.2114 - acc: 0.9148 - val_loss: 0.2846 - val_acc: 0.8847
Epoch 132/150
Epoch 00132: val_loss did not improve
 - 4s - loss: 0.1969 - acc: 0.9133 - val_loss: 0.2782 - val_acc: 0.8910
Epoch 133/150
Epoch 00133: val_loss did not improve
 - 4s - loss: 0.1815 - acc: 0.9271 - val_loss: 0.2910 - val_acc: 0.8910
Epoch 134/150
Epoch 00134: val_loss did not improve
 - 4s - loss: 0.2131 - acc: 0.9172 - val_loss: 0.3172 - val_acc: 0.8847
Epoch 135/150
Epoch 00135: val_loss did not improve
 - 4s - loss: 0.1935 - acc: 0.9263 - val_loss: 0.2774 - val_acc: 0.9128
Epoch 136/150
Epoch 00136: val_loss did not improve
 - 4s - loss: 0.2171 - acc: 0.8966 - val_loss: 0.2644 - val_acc: 0.9097
Epoch 137/150
Epoch 00137: val_loss did not improve
 - 4s - loss: 0.2002 - acc: 0.9203 - val_loss: 0.2960 - val_acc: 0.8941
Epoch 13

Epoch 44/150
Epoch 00044: val_loss did not improve
 - 4s - loss: 0.2260 - acc: 0.8985 - val_loss: 0.2965 - val_acc: 0.8750
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 3s - loss: 0.2661 - acc: 0.8859 - val_loss: 0.2888 - val_acc: 0.8781
Epoch 46/150
Epoch 00046: val_loss did not improve
 - 4s - loss: 0.2446 - acc: 0.8992 - val_loss: 0.3049 - val_acc: 0.8812
Epoch 47/150
Epoch 00047: val_loss did not improve
 - 4s - loss: 0.2376 - acc: 0.8961 - val_loss: 0.2849 - val_acc: 0.8500
Epoch 48/150
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.2488 - acc: 0.8906 - val_loss: 0.2836 - val_acc: 0.8750
Epoch 49/150
Epoch 00049: val_loss did not improve
 - 3s - loss: 0.2255 - acc: 0.9008 - val_loss: 0.2896 - val_acc: 0.8781
Epoch 50/150
Epoch 00050: val_loss did not improve
 - 4s - loss: 0.2265 - acc: 0.8867 - val_loss: 0.3037 - val_acc: 0.8781
Epoch 51/150
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.2375 - acc: 0.8953 - val_loss: 0.3154 - val_acc: 0.8688
Epoch 52/150
Epo

Epoch 00109: val_loss did not improve
 - 4s - loss: 0.2004 - acc: 0.9164 - val_loss: 0.3281 - val_acc: 0.8750
Epoch 110/150
Epoch 00110: val_loss did not improve
 - 4s - loss: 0.1926 - acc: 0.9141 - val_loss: 0.3324 - val_acc: 0.8750
Epoch 111/150
Epoch 00111: val_loss did not improve
 - 4s - loss: 0.2003 - acc: 0.9172 - val_loss: 0.3110 - val_acc: 0.8781
Epoch 112/150
Epoch 00112: val_loss did not improve
 - 4s - loss: 0.1833 - acc: 0.9203 - val_loss: 0.3416 - val_acc: 0.8781
Epoch 113/150
Epoch 00113: val_loss did not improve
 - 4s - loss: 0.1956 - acc: 0.9187 - val_loss: 0.2993 - val_acc: 0.8562
Epoch 114/150
Epoch 00114: val_loss did not improve
 - 4s - loss: 0.1891 - acc: 0.9234 - val_loss: 0.2925 - val_acc: 0.8562
Epoch 115/150
Epoch 00115: val_loss did not improve
 - 4s - loss: 0.1763 - acc: 0.9172 - val_loss: 0.3212 - val_acc: 0.8656
Epoch 116/150
Epoch 00116: val_loss did not improve
 - 4s - loss: 0.2005 - acc: 0.9211 - val_loss: 0.3038 - val_acc: 0.8688
Epoch 117/150
Epoch 00

In [19]:
with open('../features/cnn_5_aug_denoise_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
# 2364

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_5_aug_denoise_preprocess.csv', index=False)

0.232995432112
         id  is_iceberg
0  5941774d    0.322712
1  4023181e    0.215656
2  b20200e4    0.059542
3  e7f018bb    0.997717
4  4371c8c3    0.013258
